In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPool2D, AveragePooling2D, Flatten

# Helper functions and settings

In [2]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

patience = 50
epochs = 1000

earlystop = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=patience, verbose=1, mode='auto', restore_best_weights=True)

# Load MNIST

In [3]:
(train, validate, test), info = tfds.load('mnist', split=['train', 'test[:80%]', 'test[80%:]'], shuffle_files=True, with_info=True, as_supervised=True)
info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\nicol\\tensorflow_datasets\\mnist\\3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [4]:
print(f"Training set: {len(train)} examples; validation: {len(validate)} examples; reserved for testing: {len(test)} examples")

Training set: 60000 examples; validation: 8000 examples; reserved for testing: 2000 examples


In [5]:
train = train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(info.splits['train'].num_examples).batch(128).prefetch(tf.data.AUTOTUNE)
validate = validate.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).batch(128).cache().prefetch(tf.data.AUTOTUNE)
test = test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).batch(128).cache().prefetch(tf.data.AUTOTUNE)

# Fashion MNIST

In [6]:
(train_fashion, validate_fashion, test_fashion), info_fashion = tfds.load('fashion_mnist', split=['train', 'test[:80%]', 'test[80%:]'], shuffle_files=True, with_info=True, as_supervised=True)
info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\nicol\\tensorflow_datasets\\mnist\\3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [7]:
print(f"Training set: {len(train_fashion)} examples; validation: {len(validate_fashion)} examples; reserved for testing: {len(test_fashion)} examples")

Training set: 60000 examples; validation: 8000 examples; reserved for testing: 2000 examples


In [8]:
train_fashion = train_fashion.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(info.splits['train'].num_examples).batch(128).prefetch(tf.data.AUTOTUNE)
validate_fashion = validate_fashion.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).batch(128).cache().prefetch(tf.data.AUTOTUNE)
test_fashion = test_fashion.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).batch(128).cache().prefetch(tf.data.AUTOTUNE)

# original LeNET-5

In [9]:
def build_lenet5():
    model = Sequential()
    model.add(Conv2D(6, kernel_size=5, input_shape=(28, 28, 1), padding="same", activation="sigmoid"))
    model.add(AveragePooling2D())
    model.add(Conv2D(16, kernel_size=5, activation="sigmoid"))
    model.add(AveragePooling2D())
    model.add(Conv2D(120, kernel_size=5, activation="sigmoid"))
    model.add(Flatten())
    model.add(Dense(84, activation="tanh"))
    model.add(Dense(10, activation="softmax"))

    model.compile(optimizer="adam", loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=["accuracy"])
    model.summary()
    
    return model

In [10]:
lenet5 = build_lenet5()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [11]:
lenet5.fit(train, epochs=epochs, validation_data=validate, callbacks=[earlystop])

Epoch 1/1000
469/469 [==============================] - 4s 2ms/step - loss: 1.1670 - accuracy: 0.6046 - val_loss: 0.3805 - val_accuracy: 0.8834
Epoch 2/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3170 - accuracy: 0.9018 - val_loss: 0.2549 - val_accuracy: 0.9233
Epoch 3/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2277 - accuracy: 0.9280 - val_loss: 0.1892 - val_accuracy: 0.9414
Epoch 4/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1721 - accuracy: 0.9463 - val_loss: 0.1439 - val_accuracy: 0.9535
Epoch 5/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1353 - accuracy: 0.9584 - val_loss: 0.1155 - val_accuracy: 0.9635
Epoch 6/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1121 - accuracy: 0.9652 - val_loss: 0.0925 - val_accuracy: 0.9709
Epoch 7/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0956 - accuracy: 0.9694 - val_loss: 0.0770 - val_accuracy:

469/469 [==============================] - 1s 2ms/step - loss: 0.0050 - accuracy: 0.9984 - val_loss: 0.0528 - val_accuracy: 0.9883
Epoch 58/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0045 - accuracy: 0.9985 - val_loss: 0.0461 - val_accuracy: 0.9883
Epoch 59/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0065 - accuracy: 0.9977 - val_loss: 0.0470 - val_accuracy: 0.9879
Epoch 60/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.0533 - val_accuracy: 0.9874
Epoch 61/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.0449 - val_accuracy: 0.9891
Epoch 62/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 0.0436 - val_accuracy: 0.9889
Epoch 63/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.0521 - val_accuracy: 0.9884

In [12]:
print(f"Final accuracy score on test data: {accuracy_score([y for x, y in test.unbatch()], np.argmax(lenet5.predict(test), axis=1))}")

Final accuracy score on test data: 0.9875


In [13]:
lenet5.fit(train_fashion, epochs=epochs, validation_data=validate_fashion, callbacks=[earlystop])

Epoch 1/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.6816 - accuracy: 0.7662 - val_loss: 0.5353 - val_accuracy: 0.8018
Epoch 2/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.4508 - accuracy: 0.8316 - val_loss: 0.4510 - val_accuracy: 0.8382
Epoch 3/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.4072 - accuracy: 0.8503 - val_loss: 0.4249 - val_accuracy: 0.8479
Epoch 4/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3807 - accuracy: 0.8607 - val_loss: 0.4046 - val_accuracy: 0.8568
Epoch 5/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3629 - accuracy: 0.8651 - val_loss: 0.3860 - val_accuracy: 0.8649
Epoch 6/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3503 - accuracy: 0.8706 - val_loss: 0.3796 - val_accuracy: 0.8646
Epoch 7/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3386 - accuracy: 0.8749 - val_loss: 0.3962 - val_accuracy:

469/469 [==============================] - 1s 2ms/step - loss: 0.1755 - accuracy: 0.9334 - val_loss: 0.3079 - val_accuracy: 0.8994
Epoch 58/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1715 - accuracy: 0.9357 - val_loss: 0.3093 - val_accuracy: 0.8997
Epoch 59/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1692 - accuracy: 0.9358 - val_loss: 0.3124 - val_accuracy: 0.9015
Epoch 60/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1671 - accuracy: 0.9362 - val_loss: 0.3084 - val_accuracy: 0.8960
Epoch 61/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1672 - accuracy: 0.9364 - val_loss: 0.3113 - val_accuracy: 0.8996
Epoch 62/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1639 - accuracy: 0.9385 - val_loss: 0.3082 - val_accuracy: 0.8999
Epoch 63/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.1649 - accuracy: 0.9369 - val_loss: 0.3273 - val_accuracy: 0.8942

In [14]:
print(f"Final accuracy score on Fashion data: {accuracy_score([y for x, y in test_fashion.unbatch()], np.argmax(lenet5.predict(test_fashion), axis=1))}")

Final accuracy score on Fashion data: 0.8985


# Simple model improvements

* Replace sigmoid activation with relu
* Replace AvgPooling with MaxPooling

Doesn't seem to make much of a difference? Maybe a minimal one on Fashion MNIST?

In [15]:
def build_tweaked_lenet5():
    model = Sequential()
    model.add(Conv2D(6, kernel_size=5, input_shape=(28, 28, 1), padding="same", activation="relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(16, kernel_size=5, activation="relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(120, kernel_size=5, activation="relu"))
    model.add(Flatten())
    model.add(Dense(84, activation="relu"))
    model.add(Dense(10, activation="softmax"))

    model.compile(optimizer="adam", loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=["accuracy"])
    model.summary()
    
    return model

In [16]:
tweaked_model = build_tweaked_lenet5()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 6)        0         
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten_1 (Flatten)         (None, 120)              

In [17]:
tweaked_model.fit(train, epochs=epochs, validation_data=validate, callbacks=[earlystop])

Epoch 1/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3250 - accuracy: 0.9051 - val_loss: 0.0739 - val_accuracy: 0.9790
Epoch 2/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0796 - accuracy: 0.9754 - val_loss: 0.0553 - val_accuracy: 0.9839
Epoch 3/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0565 - accuracy: 0.9821 - val_loss: 0.0425 - val_accuracy: 0.9849
Epoch 4/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0473 - accuracy: 0.9852 - val_loss: 0.0354 - val_accuracy: 0.9886
Epoch 5/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0385 - accuracy: 0.9879 - val_loss: 0.0346 - val_accuracy: 0.9891
Epoch 6/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0338 - accuracy: 0.9893 - val_loss: 0.0291 - val_accuracy: 0.9912
Epoch 7/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0284 - accuracy: 0.9909 - val_loss: 0.0297 - val_accuracy:

Epoch 56: early stopping


In [18]:
print(f"Final accuracy score on test data: {accuracy_score([y for x, y in test.unbatch()], np.argmax(tweaked_model.predict(test), axis=1))}")

Final accuracy score on test data: 0.9885


In [19]:
tweaked_model.fit(train_fashion, epochs=epochs, validation_data=validate_fashion, callbacks=[earlystop])

Epoch 1/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.5019 - accuracy: 0.8203 - val_loss: 0.3861 - val_accuracy: 0.8646
Epoch 2/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3353 - accuracy: 0.8772 - val_loss: 0.3454 - val_accuracy: 0.8739
Epoch 3/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2993 - accuracy: 0.8902 - val_loss: 0.3205 - val_accuracy: 0.8871
Epoch 4/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2744 - accuracy: 0.8989 - val_loss: 0.3087 - val_accuracy: 0.8882
Epoch 5/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2563 - accuracy: 0.9058 - val_loss: 0.3045 - val_accuracy: 0.8878
Epoch 6/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2426 - accuracy: 0.9089 - val_loss: 0.3095 - val_accuracy: 0.8888
Epoch 7/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2285 - accuracy: 0.9161 - val_loss: 0.2923 - val_accuracy:

469/469 [==============================] - 1s 2ms/step - loss: 0.0375 - accuracy: 0.9860 - val_loss: 0.7840 - val_accuracy: 0.8947
Epoch 58/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0344 - accuracy: 0.9873 - val_loss: 0.8078 - val_accuracy: 0.8936
Epoch 59/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0234 - accuracy: 0.9917 - val_loss: 0.8488 - val_accuracy: 0.8972
Epoch 60/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.0229 - accuracy: 0.9922 - val_loss: 0.8628 - val_accuracy: 0.8949
Epoch 60: early stopping


In [20]:
print(f"Final accuracy score on Fashion data: {accuracy_score([y for x, y in test_fashion.unbatch()], np.argmax(tweaked_model.predict(test_fashion), axis=1))}")

Final accuracy score on Fashion data: 0.901
